In [15]:
#push!(LOAD_PATH,"../src/")
#using Revise
using ProgressBars
using ITensors
using PastaQ
using RandomMeas

In [16]:
#Perform RM in the first experiment
N = 6
circuit = randomcircuit(N, depth=2)
noisemodel1 = (1 => ("depolarizing", (p = 0,)),2 => ("depolarizing", (p = 0.05,)))
ρ1 = runcircuit(circuit;noise = noisemodel1);
ξ = firstsiteinds(ρ1;plev=0)
println(ξ)
println("state prepared in the `experiment' 1 ")

LoadError: MethodError: no method matching OpName(::String)

In [17]:
#Perform Randomized measurements
nu=200 #Number of random unitaries
NM=1000 #Number of projective measurements per unitary
u = Vector{Vector{ITensor}}()
data1 = zeros(Int8,(nu,NM,N)) #Data storage
for r in ProgressBar(1:nu, printing_delay=2)
            push!(u,get_rotations(ξ,1)) #generate random unitary
            data1[r,:,:] = get_RandomMeas(ρ1,u[r],NM) #data acquisation in simulated quantum device
end

0.0%┣                                              ┫ 0/200 [00:00<00:00, -0s/it]


LoadError: MethodError: no method matching op(::String, ::Index{Int64})

In [18]:
#Perform RM in the second experiment
noisemodel2 = (1 => ("amplitude_damping", (γ = 0.1,)),2 => ("amplitude_damping", (γ = 0.1,)))
ρ2 = runcircuit(circuit;noise = noisemodel2);
ξ2 = [firstind(ρ2[i],tags="Site",plev=0) for i in 1:N]
for i in 1:N
    replaceind!(ρ2[i],ξ2[i],ξ[i])
    replaceind!(ρ2[i],ξ2[i]',ξ[i]')
end
println("state prepared in the `experiment' 2 ")
data2 = zeros(Int8,(nu,NM,N))
for r in ProgressBar(1:nu, printing_delay=2)
                data2[r,:,:] = get_RandomMeas(ρ2,u[r],NM)
end

LoadError: MethodError: no method matching siteinds(::String, ::Int64)

In [19]:
#Build histograms for each unitary
P1 = Vector{ITensor}()
P2 = Vector{ITensor}()
for r in 1:nu
                push!(P1,get_Born(data1[r,:,:],ξ))
                push!(P2,get_Born(data2[r,:,:],ξ))
end

LoadError: BoundsError: attempt to access 2×2×2×2×2×2 Array{Int64, 6} at index [0, 0, 0, 0, 0, 0]

In [14]:
#Uncomment if you want to see an animation
# using Plots
# anim = @animate for r in 1:nu
#     y1 =  reshape(array(P1[r]),(2^N))
#     y2 =  reshape(array(P2[r]),(2^N))
#     bar(y1,label="Experiment 1")
#     bar!(y2,label="Experiment 2")
#     xlabel!("bitstring index")
#     ylabel!("Probabilities")
#     title!("Random unitary $(r)")
# end
# gif(anim,fps=0.5)

┌ Warning: /home/benoit/.julia/packages/Observers/1ausc/src/abstractdataframe/deprecated.jl no longer exists, deleted all methods
└ @ Revise ~/.julia/packages/Revise/FaTes/src/packagedef.jl:666
┌ Warning: /home/benoit/.julia/packages/Observers/1ausc/src/base/method_utils.jl no longer exists, deleted all methods
└ @ Revise ~/.julia/packages/Revise/FaTes/src/packagedef.jl:666
┌ Warning: /home/benoit/.julia/packages/ITensors/4M2ep/src/lib/ContractionSequenceOptimization/src/breadth_first_constructive.jl no longer exists, deleted all methods
└ @ Revise ~/.julia/packages/Revise/FaTes/src/packagedef.jl:666
┌ Warning: /home/benoit/.julia/packages/ITensors/4M2ep/src/lib/ContractionSequenceOptimization/src/contraction_cost.jl no longer exists, deleted all methods
└ @ Revise ~/.julia/packages/Revise/FaTes/src/packagedef.jl:666
┌ Warning: /home/benoit/.julia/packages/ITensors/4M2ep/src/lib/LazyApply/src/LazyApply.jl no longer exists, deleted all methods
└ @ Revise ~/.julia/packages/Revise/FaTes/s

In [12]:
overlaps = 0
for r in 1:nu
    global overlaps += get_overlap(P1[r],P2[r],ξ,N)/nu
end
purity1s = get_purity_hamming(data1,ξ)
purity2s = get_purity_hamming(data2,ξ)
Fs = overlaps/max(purity1s,purity2s)
println("Estimated overlap ",overlaps)
println("Estimated purity1 ",purity1s)
println("Estimated purity2 ",purity2s)
println("Estimated fidelity ",Fs)

Estimated overlap 0.50338894
Estimated purity1 0.6692999999999995
Estimated purity2 0.39987015015015015
Estimated fidelity 0.7521125653668017


In [13]:
overlap = real(inner(ρ1,ρ2))
purity1 = get_purity(ρ1)
purity2 = get_purity(ρ2)
F = overlap/max(purity1,purity2)
println("overlap ",overlap)
println("purity1 ",purity1)
println("purity2 ",purity2)
println("fidelity ",F)

overlap 0.4991973929717782
purity1 0.6477088053594152
purity2 0.40133990141787823
fidelity 0.7707126857643571
